In [21]:
import cv2
import numpy as np
import tensorflow as tf

# Load the TensorFlow Lite interpreter
interpreter = tf.lite.Interpreter(model_path=r'C:\Users\Renzo\Documents\Project Design\MoveNet\New folder\thunder.tflite')
interpreter.allocate_tensors()

# Define the function to draw keypoints and connections
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 255), 2)

# Edge definitions for connecting keypoints
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# Input video file
input_video_path = r'C:\Users\Renzo\Documents\Project Design\Datasets\test\push_up\push-up_52.mp4'

# Output video file
output_video_path = r'C:\Users\Renzo\Documents\Project Design\MoveNet\PoseOutput\output7.mp4'

# Open the input video file
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open input video.")
    exit()

# Get the properties of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create a VideoWriter object to write the output video
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

print("Processing Video...")

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the input image to match the expected input size of the model (256x256)
    img = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), 256, 256)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    # Write the processed frame to the output video
    out.write(frame)

    # Explicitly delete variables to free up memory
    del img, input_image, keypoints_with_scores

# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()

print(f"Done processing video. Output saved as {output_video_path}")


Processing Video...
Done processing video. Output saved as C:\Users\Renzo\Documents\Project Design\MoveNet\PoseOutput\output7.mp4
